In [ ]:
from IPython import get_ipython
ip = get_ipython()
ip.run_line_magic("reload_ext", "autoreload")  # these will enable module autoreloading
ip.run_line_magic("autoreload", "2")

### Load in Exports

In [ ]:
import os
from lxml import etree
import src.BrokerageExportProviders.Brokerages.IBKR.IbkrBrokerageExportProvider as ibrk
import src.ConfigurationProvider.Configuration as conf
import src.Core.FinancialEvents.GroupingProcessor.StagingGroupingProcessor as sgp
import src.Core.FinancialEvents.Utils.ProcessingUtils as pu
import arrow
import src.TaxAuthorityProvider.TaxAuthorities.Slovenia.SlovenianTaxAuthorityProvider as tap
import src.TaxAuthorityProvider.TaxAuthorities.Slovenia.Schemas.ReportTypes as rt
import src.TaxAuthorityProvider.Schemas.Configuration as cf

rootOfProject = os.getcwd()

reportsDirectory = os.path.join(rootOfProject, "imports")
generatedDirectory = os.path.join(rootOfProject, "exports")
configDirectory = os.path.join(rootOfProject, "config")

configProvider = conf.ConfigurationProvider(configDirectory)
taxPayerInfo = configProvider.getConfig()

ibkrProvider = ibrk.IbkrBrokerageExportProvider()
brokerExports = ibkrProvider.getListOfReportsAvailableForBroker(reportsDirectory)
print(brokerExports)



def loadFileAndExtractLines(file: str):  
    transactions = ibkrProvider.getBrokerEventsForReport(file)
    return transactions

brokerReports = list(map(loadFileAndExtractLines, brokerExports))
mergedReports = ibkrProvider.mergeBrokerEvents(brokerReports)

### Process Broker Exports

In [ ]:
convertedCommonFormat = ibkrProvider.transformBrokerEventsToBrokerAgnosticEvents(mergedReports)
groupingProcessor = sgp.StagingGroupingProcessor(pu.ProcessingUtils())

converted = groupingProcessor.generateGenericGroupings(convertedCommonFormat)

### Dividends Reports

In [ ]:
reportconfig = cf.TaxAuthorityConfiguration(fromDate=arrow.get("2022"), toDate=arrow.get("2023"))
provider = tap.SlovenianTaxAuthorityProvider(taxPayerInfo=taxPayerInfo, reportConfig=reportconfig)
tradeReport = provider.generateExportForTaxAuthority(reportType=rt.SlovenianTaxAuthorityReportTypes.DOH_DIV, data=converted)


tradeCsv = provider.generateSpreadsheetExport(reportType=rt.SlovenianTaxAuthorityReportTypes.DOH_DIV, data=converted)
tradeCsv.to_csv(os.path.join(generatedDirectory, 'export-dividends.csv'), index=False)

etree.ElementTree(tradeReport).write(os.path.join(generatedDirectory, "Doh_Div_3.xml"), xml_declaration=True, encoding='utf-8', pretty_print=True)

### Stock Trades

In [ ]:
reportconfig = cf.TaxAuthorityConfiguration(fromDate=arrow.get("2022"), toDate=arrow.get("2023"))

# print(convertedCommonFormat)
provider = tap.SlovenianTaxAuthorityProvider(taxPayerInfo=taxPayerInfo, reportConfig=reportconfig)
tradeReport = provider.generateExportForTaxAuthority(reportType=rt.SlovenianTaxAuthorityReportTypes.DOH_KDVP, data=converted)


tradeCsv = provider.generateSpreadsheetExport(reportType=rt.SlovenianTaxAuthorityReportTypes.DOH_KDVP, data=converted)
tradeCsv.to_csv(os.path.join(generatedDirectory, 'export-trades.csv'), index=False)

etree.ElementTree(tradeReport).write(os.path.join(generatedDirectory, "Doh_KDVP_9.xml"), xml_declaration=True, encoding='utf-8', pretty_print=True)

### Derivatives Report

In [ ]:
reportconfig = cf.TaxAuthorityConfiguration(fromDate=arrow.get("2022"), toDate=arrow.get("2023"))

# print(convertedCommonFormat)
provider = tap.SlovenianTaxAuthorityProvider(taxPayerInfo=taxPayerInfo, reportConfig=reportconfig)

tradeCsv = provider.generateSpreadsheetExport(reportType=rt.SlovenianTaxAuthorityReportTypes.D_IFI, data=converted)
tradeCsv.to_csv(os.path.join(generatedDirectory, 'export-derivatives.csv'), index=False)

tradeReport = provider.generateExportForTaxAuthority(reportType=rt.SlovenianTaxAuthorityReportTypes.D_IFI, data=converted)
etree.ElementTree(tradeReport).write(os.path.join(generatedDirectory, "D_IFI_4.xml"), xml_declaration=True, encoding='utf-8', pretty_print=True)